In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import math

# ---- ATTRIBUTES ----
ATTRIBUTES = ['Strength', 'Dexterity', 'Constitution', 'Intelligence', 'Perception', 'Willpower']
ATTR_PTS_TOTAL = 20
attr_sliders = {a: widgets.BoundedIntText(min=1, max=5, value=1, description=a, layout=widgets.Layout(width='100%')) for a in ATTRIBUTES}
attr_points_label = widgets.HTML()

def enforce_attr_limits(change=None):
    used = sum(w.value for w in attr_sliders.values())
    for a, w in attr_sliders.items():
        other = used - w.value
        max_this = min(5, ATTR_PTS_TOTAL - other)
        w.max = max(max_this, 1)
        w.min = 1
    attr_points_label.value = f"<i>Points used: {used} / {ATTR_PTS_TOTAL} &nbsp;|&nbsp; Points left: {ATTR_PTS_TOTAL - used}</i>"
    update_stats_tab()

# ---- SKILLS ----
SKILLS = [
    'Brawling', 'Dodge', 'Guns (Type)', 'Melee Weapon (Type)', 'Throwing', 'Martial Arts*',
    'First Aid', 'Medicine*', 'Occult Knowledge*', 'Trance*', 'Craft (Type)', 'Engineer (Type)*',
    'Electronics', 'Mechanic', 'Computer', 'Computer Hacking*', 'Demolitions', 'Driving (Type)',
    'Piloting (Type)', 'Sailing', 'Survival', 'Tracking', 'Traps', 'Stealth', 'Climbing', 'Running',
    'Swimming', 'Riding (Type)', 'Lockpicking (Type)', 'Notice', 'Research', 'Language (Type)',
    'Humanities (Type)', 'Science (Type)', 'Acting', 'Instruction', 'Streetwise', 'Smooth Talking',
    'Haggling', 'Beautician'
]
SKILL_PTS_TOTAL = 25

skill_widgets = []
for s in SKILLS:
    slider = widgets.BoundedIntText(value=0, min=0, max=5, layout=widgets.Layout(width='60px'))
    skill_widgets.append((widgets.Label(s, layout=widgets.Layout(width='100%')), slider))
skill_sliders = {s: w for s, w in zip(SKILLS, [w for l, w in skill_widgets])}
skill_points_label = widgets.HTML()

def enforce_skill_limits(change=None):
    used = sum(w.value for w in skill_sliders.values())
    for s, w in skill_sliders.items():
        other = used - w.value
        max_this = min(5, SKILL_PTS_TOTAL - other)
        w.max = max(max_this, 0)
        w.min = 0
    skill_points_label.value = f"<i>Points used: {used} / {SKILL_PTS_TOTAL} &nbsp;|&nbsp; Points left: {SKILL_PTS_TOTAL - used}</i>"
    update_stats_tab()

half = (len(skill_widgets) + 1) // 2
col1 = [widgets.HBox([label, box], layout=widgets.Layout(width='100%')) for label, box in skill_widgets[:half]]
col2 = [widgets.HBox([label, box], layout=widgets.Layout(width='100%')) for label, box in skill_widgets[half:]]
skill_tab = widgets.VBox([
    widgets.HBox([widgets.VBox(col1, layout=widgets.Layout(width='100%')), widgets.VBox(col2, layout=widgets.Layout(width='100%'))],
                 layout=widgets.Layout(width='100%')),
    skill_points_label
], layout=widgets.Layout(width='100%'))

# ---- QUALITIES & DRAWBACKS ----
QUALITIES = [
    ('Fast Reaction Time', 2), ('Hard to Kill', 1), ('Situational Awareness', 2),
    ('Nerves of Steel', 3), ('Acute Senses', 1), ('Photographic Memory', 2),
    ('Resistance', 2), ('Attractiveness', 1), ('Charisma', 2), ('Natural Toughness', 2),
    ('Ambidextrous', 1), ('Resources', 1), ('Contacts', 1), ('Fame', 1), ('Jack-of-All-Trades', 2)
]
quality_checks = {q[0]: widgets.Checkbox(description=f"{q[0]} (+{q[1]})", layout=widgets.Layout(width='100%')) for q in QUALITIES}
quality_costs = [q[1] for q in QUALITIES]

DRAWBACKS = [
    ('Addiction', 1), ('Claustrophobia', 1), ('Cowardly', 1), ('Cruel', 1),
    ('Delusions', 1), ('Emotional Problems', 1), ('Honorable', 1), ('Humorless', 1),
    ('Lazy', 2), ('Obsession', 2), ('Paranoid', 2), ('Recurring Nightmares', 1),
    ('Reckless', 2), ('Showoff', 2), ('Talentless', 2), ('Zealot', 3),
    ('Impaired Sense', 1), ('Physical Disability', 2), ('Weak Stomach', 1),
    ('Adversary', 1), ('Dependents', 2), ('Honest to a Fault', 1), ('Minority', 1),
    ('Secret', 1), ('Status', 1), ('Accursed', 1), ('Old Soul', 3), ('Bad Luck', 1)
]
drawback_checks = {d[0]: widgets.Checkbox(description=f"{d[0]} (+{d[1]})", layout=widgets.Layout(width='100%')) for d in DRAWBACKS}
drawback_costs = [d[1] for d in DRAWBACKS]
QUAL_PTS_START = 10
DRAWBACK_PTS_LIMIT = 10
qualities_points_label = widgets.HTML()
drawbacks_points_label = widgets.HTML()

def enforce_drawback_limits(change=None):
    total_drawbacks = sum(w.value*c for w,c in zip(drawback_checks.values(),drawback_costs))
    if total_drawbacks > DRAWBACK_PTS_LIMIT:
        drawn = [(i, w) for i, (d, w) in enumerate(drawback_checks.items()) if w.value]
        for i, w in reversed(drawn):
            if total_drawbacks > DRAWBACK_PTS_LIMIT:
                w.unobserve(enforce_drawback_limits, 'value')
                w.value = False
                w.observe(enforce_drawback_limits, 'value')
                total_drawbacks = sum(w.value*c for w,c in zip(drawback_checks.values(),drawback_costs))
    drawbacks_points_label.value = f"<i>Drawback points used: {total_drawbacks} / {DRAWBACK_PTS_LIMIT} | Points for Qualities: {QUAL_PTS_START} + {total_drawbacks}</i>"
    enforce_quality_limits()
    update_stats_tab()

def enforce_quality_limits(change=None):
    total_drawbacks = sum(w.value*c for w,c in zip(drawback_checks.values(),drawback_costs))
    qual_budget = QUAL_PTS_START + total_drawbacks
    used = sum(w.value*c for w,c in zip(quality_checks.values(),quality_costs))
    if used > qual_budget:
        drawn = [(i, w, c) for i, ((q, w), c) in enumerate(zip(quality_checks.items(), quality_costs)) if w.value]
        running = used
        for i, w, c in reversed(drawn):
            if running > qual_budget:
                w.unobserve(enforce_quality_limits, 'value')
                w.value = False
                w.observe(enforce_quality_limits, 'value')
                running -= c
    used = sum(w.value*c for w,c in zip(quality_checks.values(),quality_costs))
    qualities_points_label.value = f"<i>Points used: {used} / {qual_budget} &nbsp;|&nbsp; Points left: {qual_budget - used}</i>"
    update_stats_tab()

# ---------- ACTIVE WEAPONS TABLE ----------
weapon_rows = 4
weapon_cols = ["Name", "Damage", "Attributes", "Ammo", "Traits", "Encumbrance"]
weapon_table = [
    [widgets.Text(placeholder=col, layout=widgets.Layout(width='100%')) for col in weapon_cols]
    for _ in range(weapon_rows)
]
weapon_total_enc = widgets.HTML(value="<b>Total Weapon Encumbrance: 0</b>")

def calc_weapon_enc(change=None):
    total = 0
    for row in weapon_table:
        try:
            val = float(row[5].value)
        except:
            val = 0
        total += val
    weapon_total_enc.value = f"<b>Total Weapon Encumbrance: {total}</b>"
    update_stats_tab()

for row in weapon_table:
    row[5].observe(calc_weapon_enc, names='value')

def weapons_box():
    header = widgets.HBox([widgets.HTML(f"<b>{col}</b>", layout=widgets.Layout(width='100%')) for col in weapon_cols])
    rows = [widgets.HBox(row, layout=widgets.Layout(width='100%')) for row in weapon_table]
    return widgets.VBox([header] + rows + [weapon_total_enc], layout=widgets.Layout(width='100%'))

# ---------- ACTIVE ARMOR TABLE ----------
armor_rows = 4
armor_cols = ["Name", "Armor Value", "Encumbrance"]
armor_table = [
    [widgets.Text(placeholder=col, layout=widgets.Layout(width='100%')) for col in armor_cols]
    for _ in range(armor_rows)
]
armor_total_enc = widgets.HTML(value="0")
armor_total_val = widgets.HTML(value="0")

def calc_armor_enc(change=None):
    total_enc = 0
    total_val = 0
    for row in armor_table:
        try:
            enc = float(row[2].value)
        except:
            enc = 0
        try:
            val = float(row[1].value)
        except:
            val = 0
        total_enc += enc
        total_val += val
    armor_total_enc.value = f"{total_enc}"
    armor_total_val.value = f"{total_val}"
    update_stats_tab()

for row in armor_table:
    row[2].observe(calc_armor_enc, names='value')
    row[1].observe(calc_armor_enc, names='value')

def armor_box():
    header = widgets.HBox([widgets.HTML(f"<b>{col}</b>", layout=widgets.Layout(width='100%')) for col in armor_cols])
    rows = [widgets.HBox(row, layout=widgets.Layout(width='100%')) for row in armor_table]
    return widgets.VBox([header] + rows, layout=widgets.Layout(width='100%'))

# ---------- INVENTORY TABLE (8 rows x 6 cols, no scrollbars) ----------
inv_rows = 8
inv_cols = ['Item Name', 'EV', 'Amount', 'Item Name', 'EV', 'Amount']
inventory_table = [
    [widgets.Text(placeholder=col, layout=widgets.Layout(width='100%')) for col in inv_cols]
    for _ in range(inv_rows)
]
inv_total_enc = widgets.HTML(value="0")

def calc_inventory_enc(change=None):
    total = 0
    for row in inventory_table:
        for idx in [1, 4]:  # EV columns
            try:
                val = float(row[idx].value)
                amt = float(row[idx + 1].value) if row[idx + 1].value else 1
            except:
                val = 0
                amt = 1
            total += val * amt
    inv_total_enc.value = f"{total}"
    update_stats_tab()

for row in inventory_table:
    for idx in [1, 2, 4, 5]:
        row[idx].observe(calc_inventory_enc, names='value')

def inventory_box():
    header = widgets.HBox([widgets.HTML(f"<b>{col}</b>", layout=widgets.Layout(width='100%')) for col in inv_cols])
    rows = [widgets.HBox(row, layout=widgets.Layout(width='100%')) for row in inventory_table]
    return widgets.VBox([header] + rows + [widgets.HTML('<b>Total Inventory Encumbrance:</b>'), inv_total_enc],
                        layout=widgets.Layout(width='100%'))

# ---------- PLAYER STATS TAB ----------
player_stats_out = widgets.Output()

def update_stats_tab(change=None):
    with player_stats_out:
        clear_output()
        Str = attr_sliders['Strength'].value
        Dex = attr_sliders['Dexterity'].value
        Con = attr_sliders['Constitution'].value
        Int = attr_sliders['Intelligence'].value
        Per = attr_sliders['Perception'].value
        Will = attr_sliders['Willpower'].value

        life = (Str+Con)*4+10
        endu = (Str+Con+Will)*3
        ess  = (Will+Con)*2
        speed = math.floor((Str+Dex)/2)

        armor_val = armor_total_val.value
        armor_enc = armor_total_enc.value
        inv_enc = inv_total_enc.value
        try:
            weap_enc = float(weapon_total_enc.value.split(":")[1].replace("</b>", "").strip())
        except:
            weap_enc = 0
        try:
            total_enc = float(armor_enc) + float(inv_enc) + weap_enc
        except:
            total_enc = 0

        sel_skills = [f"{k} <b>({v.value})</b>" for k,v in skill_sliders.items() if v.value>0]
        sel_qual = [k for k,v in quality_checks.items() if v.value]
        sel_draw = [k for k,v in drawback_checks.items() if v.value]

        # Inventory list
        inv_list = []
        for row in inventory_table:
            for i in [0, 3]:
                if row[i].value:
                    name = row[i].value
                    amt = row[i+2].value if (i+2)<6 else ""
                    ev = row[i+1].value if (i+1)<6 else ""
                    inv_list.append(f"{name} x{amt or 1} (EV {ev or 0})")

        # Weapons list
        weapons_list = []
        for row in weapon_table:
            if row[0].value:
                weapons_list.append(f"<tr><td>{row[0].value}</td><td>{row[1].value}</td><td>{row[2].value}</td><td>{row[3].value}</td><td>{row[4].value}</td><td>{row[5].value}</td></tr>")

        # Armor list
        armor_list = []
        for row in armor_table:
            if row[0].value:
                armor_list.append(f"<tr><td>{row[0].value}</td><td>{row[1].value}</td><td>{row[2].value}</td></tr>")

        html = f"""
        <style>
        .stats-table, .stats-table th, .stats-table td {{
            border:1px solid #aaa;
            padding:2px 8px;
            font-size:14px;
            text-align:center !important;
        }}
        .stats-table {{
            border-collapse:collapse;
            width:100%;
            margin-bottom:8px;
        }}
        .section-title {{
            font-weight:bold;
            font-size:16px;
            margin-top:12px;
            color:#305070
        }}
        @media (max-width: 600px) {{
            .stats-table, .stats-table th, .stats-table td {{font-size:12px; padding:2px 2px;}}
            .section-title {{font-size:14px;}}
        }}
        </style>
        <div class="section-title">Character Attributes</div>
        <table class="stats-table">
            <tr><th>Strength</th><th>Dexterity</th><th>Constitution</th>
                <th>Intelligence</th><th>Perception</th><th>Willpower</th></tr>
            <tr><td>{Str}</td><td>{Dex}</td><td>{Con}</td><td>{Int}</td><td>{Per}</td><td>{Will}</td></tr>
        </table>
        <div class="section-title">Core Stats</div>
        <table class="stats-table">
            <tr><th>Life Points</th><th>Endurance</th><th>Essence Pool</th>
                <th>Base Speed</th><th>Armor Value</th><th>Total Encumbrance</th></tr>
            <tr><td>{life}</td><td>{endu}</td><td>{ess}</td><td>{speed}</td><td>{armor_val}</td><td>{total_enc:.2f}</td></tr>
        </table>
        <div class="section-title">Selected Skills</div>
        <div>{"<br>".join(sel_skills) if sel_skills else "-"}</div>
        <div class="section-title">Qualities</div>
        <div>{"<br>".join(sel_qual) if sel_qual else "-"}</div>
        <div class="section-title">Drawbacks</div>
        <div>{"<br>".join(sel_draw) if sel_draw else "-"}</div>
        <div class="section-title">Weapons</div>
        <table class="stats-table">
            <tr><th>Name</th><th>Damage</th><th>Attributes</th><th>Ammo</th><th>Traits</th><th>Enc.</th></tr>
            {''.join(weapons_list) if weapons_list else '<tr><td colspan="6">-</td></tr>'}
        </table>
        <div class="section-title">Armor</div>
        <table class="stats-table">
            <tr><th>Name</th><th>Armor Value</th><th>Encumbrance</th></tr>
            {''.join(armor_list) if armor_list else '<tr><td colspan="3">-</td></tr>'}
        </table>
        <div class="section-title">Inventory</div>
        <div>{"<br>".join(inv_list) if inv_list else "-"}</div>
        """
        display(widgets.HTML(value=html))

# ---- Attach observers ----
for w in attr_sliders.values():
    w.observe(enforce_attr_limits, 'value')
for w in skill_sliders.values():
    w.observe(enforce_skill_limits, 'value')
for w in quality_checks.values():
    w.observe(enforce_quality_limits, 'value')
for w in drawback_checks.values():
    w.observe(enforce_drawback_limits, 'value')
for row in armor_table:
    row[0].observe(update_stats_tab, 'value')
    row[1].observe(update_stats_tab, 'value')
    row[2].observe(update_stats_tab, 'value')
for row in weapon_table:
    row[0].observe(update_stats_tab, 'value')
    row[1].observe(update_stats_tab, 'value')
    row[2].observe(update_stats_tab, 'value')
    row[3].observe(update_stats_tab, 'value')
    row[4].observe(update_stats_tab, 'value')
    row[5].observe(update_stats_tab, 'value')
for row in inventory_table:
    for i in [0, 1, 2, 3, 4, 5]:
        row[i].observe(update_stats_tab, 'value')

# ---- Tabs ----
attr_tab = widgets.VBox(list(attr_sliders.values()) + [attr_points_label], layout=widgets.Layout(width='100%'))
qual_tab = widgets.VBox(list(quality_checks.values()) + [qualities_points_label], layout=widgets.Layout(width='100%'))
drawback_tab = widgets.VBox(list(drawback_checks.values()) + [drawbacks_points_label], layout=widgets.Layout(width='100%'))

tab = widgets.Tab()
tab.children = [
    attr_tab,
    skill_tab,
    qual_tab,
    drawback_tab,
    weapons_box(),
    armor_box(),
    inventory_box(),
    player_stats_out
]
tab.set_title(0, 'Attributes')
tab.set_title(1, 'Skills')
tab.set_title(2, 'Qualities')
tab.set_title(3, 'Drawbacks')
tab.set_title(4, 'Weapons')
tab.set_title(5, 'Armor')
tab.set_title(6, 'Inventory')
tab.set_title(7, 'Player Stats')

display(tab)

# ---- Initial calculations ----
enforce_attr_limits()
enforce_skill_limits()
enforce_quality_limits()
enforce_drawback_limits()
calc_armor_enc()
calc_inventory_enc()
calc_weapon_enc()
update_stats_tab()
